In [23]:
# 匯入套件
from bs4 import BeautifulSoup as bs

# 強制等待 (執行期間休息一下)
from time import sleep

# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

#處理瀏覽器的警告視窗
from selenium.webdriver.common.alert import Alert



# 使用 Chrome 的 WebDriver
'''
my_service = Service(executable_path="./chromedriver.exe")
driver = webdriver.Chrome(service=my_service)
'''

# 補充: 若沒有特別設定，只要電腦有安裝 Chrome，就可以直接使用
driver = webdriver.Chrome()

# 開啟神腦福利手機頁面
driver.get("https://online.senao.com.tw/Category/2/GP2207-001?sort=1&page=1")

# 等待元素出現的時間，這裡設定最多等待 20 秒鐘
wait = WebDriverWait(driver, 20)

# 等待新頁面加載完成，假設列表標題元素會在新頁面中加載
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "li.next")))

# 取得檢視原始碼的內容 (page_source 取得的 html，是動態的、使用者操作過後的結果)
html = driver.page_source

# 印出 html (也可以跟 Beautifulsoup 整合)
# print(html)

# 指定 lxml 作為解析器
soup = bs(html, "lxml")

# 等待彈出窗口出現
try:
    # 切換到警告框並接受或關閉
    alert = Alert(driver)
    alert.accept()  # 點擊確定
    # 或者使用 alert.dismiss() 來取消
except:
    print("沒有彈出警告框")


# 建立 list 來放置列表資訊
list_posts = []


沒有彈出警告框


#### soup + selenium

In [ ]:
# 清空放置列表資訊的變數
list_posts.clear()

import json

count=1
while True:
    sleep(1)
    print(count)
    html = driver.page_source
    soup = bs(html, "lxml")
    for b in soup.select('a.cycle-title-link'):
        print(b.get_text())
        #print('https://online.senao.com.tw' + b['href']) 
        list_posts.append({
            'title': b.get_text(),
            'link': 'https://online.senao.com.tw' + b['href']
        })

    with open('posts_data.json', 'w', encoding='utf-8') as f:
        json.dump(list_posts, f, ensure_ascii=False, indent=4)
    
    count +=1
    sleep(0.5)
    try :
        fin = driver.find_element(By.CSS_SELECTOR, "li.next.disabled")
        break
    except :
        driver.find_element(By.CSS_SELECTOR, "li.next > a").click()


#### only selenium

In [ ]:
count=1
while True:
    sleep(1)
    print(count)
    for item in driver.find_elements(By.CSS_SELECTOR, 'a.cycle-title-link'):
        b = item.get_attribute('text')
        print(b)
        print(item.get_attribute('href')) 
        list_posts.append({
            'title': item.get_attribute('text'),
            'link': item.get_attribute('href')
        })
    
    count +=1
    sleep(0.5)
    try :
        fin = driver.find_element(By.CSS_SELECTOR, "li.next.disabled")
        break
    except :
        driver.find_element(By.CSS_SELECTOR, "li.next > a").click()

進網頁爬蟲

In [ ]:
import json
import requests as req
from bs4 import BeautifulSoup as bs


# res = req.get('https://online.senao.com.tw/Mart/1369847') 
# soup = bs(res.text, "lxml")

# product_title = soup.select("h2.product-title")[0].get_text().strip()
# product_desc = soup.select("h3.product-desc")[0].get_text().strip()
# product_price = soup.select("b.fn-xl")[0].get_text()
# print(f"product_title: {product_title}\nproduct_desc: {product_desc}\nproduct_price: {product_price}")


with open('posts_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

product_info = []

product_info.clear()

# 迴圈每個項目，使用 link 去抓取網頁
for item in data:
    url = item['link'] # 從 JSON 中取出 link 欄位
    res = req.get(url) # 用 requests 的 get 方法把網頁抓下來
    soup = bs(res.text, "lxml") # 指定 lxml 作為解析器

    product_title = soup.select("h2.product-title")[0].get_text().strip()
    product_desc = soup.select("h3.product-desc")[0].get_text().strip()
    product_price = soup.select("b.fn-xl")[0].get_text()
    print(f"product_title: {product_title}\nproduct_desc: {product_desc}\nproduct_price: {product_price}")

    product_info.append({
        "product_title": product_title,
        "product_desc": product_desc,
        "product_price": product_price
        })

with open('product_info.json', 'w', encoding='utf-8') as f:
    json.dump(product_info, f, ensure_ascii=False, indent=4)  
    

In [29]:
product_info

[{'product_title': 'iPhone SE 128GB(2022)【A級二手機 六個月保固】',
  'product_desc': '★4.7 吋 Retina HD 顯示器★A15 仿生晶片★支援無線充電、IP67 等級防潑抗水★主鏡頭：1200 萬像素廣角相機★前鏡頭：700 萬像素相機★Touch ID 、Apple Pay',
  'product_price': '$8,490'}]